### Move data from google sheets to s3

In [1]:
import pandas as pd

In [2]:
import gspread

In [3]:
from gspread_dataframe import get_as_dataframe

In [6]:
gc = gspread.service_account()

In [10]:
gc.list_spreadsheet_files()

[{'id': '19acmExRyj-5tn2aATmIfoh1oOgFis-r2j-wOPVfkrCE',
  'name': 'Workout log',
  'createdTime': '2015-01-02T17:20:02.687Z',
  'modifiedTime': '2022-10-28T14:55:39.111Z'}]

In [11]:
sh = gc.open("Workout log")

In [14]:
worksheet = sh.get_worksheet(0)

In [19]:
df = get_as_dataframe(worksheet, 
                      parse_dates=True, 
                      usecols=[0,1,2], 
                      skiprows=None, 
                      #header=True
                     )

In [21]:
df.head()

,Date,Miles,Notes
0,7/20/2013,5.5,NaN
1,7/21/2013,6.5,NaN
2,7/22/2013,NaN,NaN
3,7/23/2013,5.5,NaN
4,7/24/2013,7.0,NaN


In [22]:
df.shape

(4087, 3)

In [23]:
df.tail()

,Date,Miles,Notes
4082,NaN,NaN,NaN
4083,NaN,NaN,NaN
4084,NaN,NaN,NaN
4085,NaN,NaN,NaN
4086,NaN,NaN,NaN


In [24]:
print(df.shape)
df = df[pd.notnull(df['Date'])]
print(df.shape)

(4087, 3)
(3391, 3)


In [25]:
df.tail()

,Date,Miles,Notes
3386,10/27/2022,NaN,lift push
3387,10/28/2022,NaN,NaN
3388,10/29/2022,NaN,NaN
3389,10/30/2022,NaN,NaN
3390,10/31/2022,NaN,NaN


### Write csv to s3

In [27]:
import boto3
from io import StringIO

In [28]:
bucket = "jeremyruns.com"
file_name = "daily_run_log.csv"

In [29]:
csv_buffer = StringIO()
df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, file_name).put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'F57FRXS4AE2JQSTB',
  'HostId': 'dfnBWy73E+soZOP4bQdVCf1nMFVe8QZgjfqNkoCjqpfNblC8v/7QRQfaLiz648omQI+bdox8XhY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'dfnBWy73E+soZOP4bQdVCf1nMFVe8QZgjfqNkoCjqpfNblC8v/7QRQfaLiz648omQI+bdox8XhY=',
   'x-amz-request-id': 'F57FRXS4AE2JQSTB',
   'date': 'Fri, 28 Oct 2022 15:13:13 GMT',
   'etag': '"02aa2324e47d60ef719990cf80b72812"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"02aa2324e47d60ef719990cf80b72812"'}